# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Знакомство с данными 

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

import numpy as np
from sklearn.dummy import DummyClassifier

In [22]:
df = pd.read_csv('/datasets/users_behavior.csv')
display(df.head())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Так как мы взяли обработанный датасет, то предобработка данных не нужна. Начнем строить модель для задачи классификации - тариф Ультра или Смарт?

## Делим выборку

Выделим обучающий набор данных в переменных features (признаки) и target (целевой признак).Тариф это то, что модель должна предсказать. 

In [23]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Разделим наши датасеты на обучающую, тестовую и валидационную выборки. Сначала разделим на обучающую и тестовую (60% и 40% соответственно), затем тестовую выборку разделим на тестовую и валидационную по 20% так, чтобы в целом получилось соотношение 3:1:1.

In [24]:
(features_train,
 features_test,
 target_train,
 target_test) = train_test_split(features, target, test_size = 0.4, random_state = 12345)

In [25]:
(features_valid,
 features_test,
 target_valid,
 target_test) = train_test_split(features_test, target_test, test_size = 0.2, random_state = 12345)

## Исследование модели 

**Random forest** 

Запишем модель в переменной, указав гиперпараметры. Зафиксируем псевдослучайность параметром random_state.
Чтобы управлять количеством деревьев в лесу, пропишем гиперпараметр n_estimator и глубину дерева depth.
Затем обучим модель и она будет готова предсказывать методом predict(). И наконец проверим правильность модели функцией accuracy_score(). 

In [26]:
best_model = None
best_accuracy = 0

for est in range(2,51):
    for depth in range(2,25):
        model = RandomForestClassifier(random_state = 123, n_estimators = est, max_depth = depth)
        model.fit(features_train,target_train)
        prediction_valid = model.predict(features_valid)
        accuracy = accuracy_score(prediction_valid, target_valid)
        if best_accuracy < accuracy:
            best_model = model
            best_accuracy = accuracy

print("Accuracy наилучшей модели на валидационной выборке:", best_accuracy)

Accuracy наилучшей модели на валидационной выборке: 0.8073929961089494


**Decision Tree**

Аналогичная процедура обучения модели. В гиперпараметрах указываем псевдослучайность и глубину дерева. 

In [27]:
best_model_DT = None
best_accuracy_DT = 0

for depth in range(2,50):
    model_DT = DecisionTreeClassifier(random_state = 123, max_depth = depth)
    model_DT.fit(features_train,target_train)
    prediction_valid_DT = model_DT.predict(features_valid)
    accuracy_DT = accuracy_score(prediction_valid_DT, target_valid)
    if best_accuracy_DT < accuracy_DT:
        best_model_DT = model_DT
        best_accuracy_DT = accuracy_DT
        
        
print("Accuracy наилучшей модели на валидационной выборке:", best_accuracy_DT)

Accuracy наилучшей модели на валидационной выборке: 0.7889105058365758


**LogisticRegression**

Меняли разные гиперпараметры, разницы в accuracy не было никакой. Solver мы используем для задачи оптимизации. А значение lbfgs для наборов данных с несколькими классами.

In [28]:
model_LR = LogisticRegression(random_state = 1234, solver = 'lbfgs' )
model_LR.fit(features_train,target_train)
prediction_valid_LR = model_LR.predict(features_valid)
accuracy_LR = accuracy_score(prediction_valid_LR, target_valid)

print("Accuracy модели логистической регрессии на валидационной выборке:", accuracy_LR)

Accuracy модели логистической регрессии на валидационной выборке: 0.6984435797665369


In [29]:
print(f"Logistic Regression: {accuracy_LR}")
print(f"Random Forest: {best_accuracy}")
print(f"Decision Tree: {best_accuracy_DT}")

Logistic Regression: 0.6984435797665369
Random Forest: 0.8073929961089494
Decision Tree: 0.7889105058365758


**Выводы**

В результате исследования, мы выявили, что наиболее точные результаты дает модель Random Forest с значением Accuracy 0.80. Логистическая регрессия дает самый маленький процент точности - 69%. 
Проверим модель на тестовой выборке, которую выделили в начале проекта. 

## Проверка модели на тестовой выборке

Обученные модели готовы предсказывать на тестовой выборке. 

In [30]:
prediction_test_LR = model_LR.predict(features_test)
prediction_test_RF = best_model.predict(features_test)
prediction_test_DT = best_model_DT.predict(features_test)

Проверим точность.

In [31]:
accuracy_LR_test = accuracy_score(prediction_test_LR, target_test)
accuracy_RF_test = accuracy_score(prediction_test_RF, target_test)
accuracy_DT_test = accuracy_score(prediction_test_DT, target_test)

In [32]:
print(f"Logistic Regression: {accuracy_LR_test}")
print(f"Random Forest: {accuracy_RF_test}")
print(f"Decision Tree: {accuracy_DT_test}")

Logistic Regression: 0.6937984496124031
Random Forest: 0.7984496124031008
Decision Tree: 0.7945736434108527


In [33]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train,target_train)
dummy_clf.predict(features_train)
dummy_clf.score(features_train,target_train)


0.6924273858921162

In [34]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_test,target_test)
dummy_clf.predict(features_test)
dummy_clf.score(features_test,target_test)


0.689922480620155

**Выводы**

Снова Random Forest дает более точные предсказания, однако тестовая выборка также показала высокую точность предсказания в модели Decision Tree. Логистиеская Регрессия опять нам показывает тот же процент точности - 69%.

Получается, что модель Случайный Лес является самой подходящей моделью для задачи классификации, которая выберет подходящий тариф.

Проверка модели на адекватность показала, что точность модели была понижена в среднем на 10%. В целом, точность классификатора не выглядит случайной, что может означать: гиперпараметры установлены правильно, набор данных, используемый классификатором, является сбаланированным и т.д. 